<a href="https://colab.research.google.com/github/khairav/MachinLearning-Mini_Project/blob/main/submission_2_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
!mkdir -p ~/Dicoding
%cd /content/drive/MyDrive/MachineLearningProject/Dicoding


/content/drive/MyDrive/MachineLearningProject/Dicoding


# import library

In [6]:
import numpy as np
import pandas as pd
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import tensorflow as tf

#import Dataset


In [7]:
# install kaggle package
!pip install -q kaggle
# upload kaggle.json
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"denidiana","key":"936b134a019e93123411ec2a6bff8ebb"}'}

In [9]:
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [11]:
# test kaggle dataset list
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        195           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        323           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        953           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       5936           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      49145            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge      12820           False  
connectx

Data set from compettion sumanthvrao/daily-climate-time-series-data

In [46]:
# download dataset, choose 'copy api command' from kaggle dataset
!kaggle datasets download -d sherrytp/airline-delay-analysis

100% 2.19G/2.19G [00:29<00:00, 99.6MB/s]



In [54]:
!unzip airline-delay-analysis.zip 
%ls

Archive:  airline-delay-analysis.zip
  inflating: airline delay analysis/20.csv  
  inflating: airline delay analysis/2009.csv  
  inflating: airline delay analysis/2010.csv  
  inflating: airline delay analysis/2011.csv  
  inflating: airline delay analysis/2012.csv  
  inflating: airline delay analysis/2013.csv  
  inflating: airline delay analysis/2014.csv  
  inflating: airline delay analysis/2015.csv  
  inflating: airline delay analysis/2016.csv  
  inflating: airline delay analysis/2017.csv  
  inflating: airline delay analysis/2018.csv  
  inflating: airline delay analysis/2019.csv  
'airline delay analysis'/   airline-delay-analysis.zip   Dicoding/


In [70]:
#/content/drive/MyDrive/MachineLearningProject/Dicoding/Dicoding/airline delay analysis/20.csv
%cd /content/drive/MyDrive/MachineLearningProject/Dicoding/Dicoding
data_train = pd.read_csv('airline delay analysis/20.csv')
data_train.head()

/content/drive/MyDrive/MachineLearningProject/Dicoding/Dicoding


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2020-01-01,WN,5888,ONT,SFO,1851.0,41.0,44.0,1935.0,2049.0,4.0,2053.0,68.0,74.0,363.0,8.0,0.0,27.0,0.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-01,WN,6276,ONT,SFO,1146.0,-4.0,13.0,1159.0,1310.0,8.0,1318.0,-2.0,71.0,363.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01,WN,4598,ONT,SJC,2016.0,-4.0,8.0,2024.0,2121.0,3.0,2124.0,-6.0,57.0,333.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-01,WN,4761,ONT,SJC,1350.0,10.0,10.0,1400.0,1503.0,2.0,1505.0,10.0,63.0,333.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-01,WN,5162,ONT,SJC,916.0,1.0,6.0,922.0,1019.0,4.0,1023.0,-12.0,57.0,333.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
data_train.columns

Index(['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 27'],
      dtype='object')

# Cleansing data

In [72]:
#cek data 
data_train.isnull().sum()

FL_DATE                      0
OP_CARRIER                   0
OP_CARRIER_FL_NUM            0
ORIGIN                       0
DEST                         0
CRS_DEP_TIME            263935
DEP_TIME                263990
DEP_DELAY               264407
TAXI_OUT                264407
WHEELS_OFF              265285
WHEELS_ON               265285
TAXI_IN                 265285
CRS_ARR_TIME            268764
ARR_TIME                268764
ARR_DELAY                    0
CANCELLED              2295172
CANCELLATION_CODE      2295172
DIVERTED               2295172
CRS_ELAPSED_TIME       2295172
ACTUAL_ELAPSED_TIME    2295172
AIR_TIME               2547574
DISTANCE               2547574
CARRIER_DELAY          2547574
WEATHER_DELAY          2547574
NAS_DELAY              2547574
SECURITY_DELAY         2547574
LATE_AIRCRAFT_DELAY    2547574
Unnamed: 27            2547574
dtype: int64

In [74]:
data_train.describe()

,OP_CARRIER_FL_NUM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
count,2.547574e+06,2.283639e+06,2.283584e+06,2.283167e+06,2.283167e+06,2.282289e+06,2.282289e+06,2.282289e+06,2.278810e+06,2.278810e+06,2.547574e+06,252402.000000,252402.000000,252402.000000,252402.000000,252402.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,2.598624e+03,1.325070e+03,3.395949e+00,1.584696e+01,1.349064e+03,1.472520e+03,6.961088e+00,1.478037e+03,-4.633769e+00,1.099997e+02,7.820224e+02,25.204392,3.830259,14.500170,0.096746,20.690953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.819841e+03,4.881390e+02,3.880338e+01,8.726017e+00,4.880447e+02,5.093092e+02,5.198871e+00,5.124247e+02,4.091384e+01,6.836834e+01,5.644905e+02,75.171648,31.989144,32.972045,3.524983,50.927167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000e+00,1.000000e+00,-8.000000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.170000e+02,7.000000e+00,2.900000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.033000e+03,9.200000e+02,-7.000000e+00,1.100000e+01,9.340000e+02,1.058000e+03,4.000000e+00,1.102000e+03,-2.000000e+01,6.000000e+01,3.680000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2.172000e+03,1.322000e+03,-4.000000e+00,1.400000e+01,1.334000e+03,1.506000e+03,6.000000e+00,1.510000e+03,-1.100000e+01,9.300000e+01,6.320000e+02,0.000000,0.000000,2.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,4.087000e+03,1.731000e+03,0.000000e+00,1.800000e+01,1.744000e+03,1.905000e+03,8.000000e+00,1.910000e+03,-1.000000e+00,1.400000e+02,1.020000e+03,23.000000,0.000000,19.000000,0.000000,22.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,9.888000e+03,2.400000e+03,2.814000e+03,1.890000e+02,2.400000e+03,2.400000e+03,1.940000e+02,2.400000e+03,2.794000e+03,6.980000e+02,5.095000e+03,2560.000000,1525.000000,1462.000000,1185.000000,2228.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Data Format¶
MONTH - Month
DAY_OF_MONTH - Day of Month
DAY_OF_WEEK - Day of Week
OP_UNIQUE_CARRIER - Unique Carrier Code
ORIGIN - Origin airport location
DEST - Destination airport location
DEP_TIME - Actual Departure Time (local time: hhmm)
DEP_DEL15 - Departure Delay Indicator, 15 Minutes or More (1=Yes, 0=No) [TARGET VARIABLE]
DISTANCE - Distance between airports (miles)

In [75]:
data_train['Unnamed: 27'].unique()

array([nan])

In [76]:
data_train.drop('Unnamed: 27', axis=1, inplace=True)

In [77]:
data_train.columns

Index(['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'],
      dtype='object')

In [78]:
data_train.shape

(2547574, 27)

In [63]:
data_train.rename(columns={'DEP_DELAY':'is_delay'}, inplace=True)

In [79]:
print(f'\'DEP_DELAY\' missing values are {100*2547574/data_train.shape[0]}%')
print(f'\'DEP_TIME\' missing values are {100*2547574/data_train.shape[0]}%')

'DEP_DELAY' missing values are 100.0%
'DEP_TIME' missing values are 100.0%


In [ ]:
# Replace using median 
median = flight['DEP_DELAY'].median()
flight['DEP_DELAY'].fillna(median, inplace=True)

In [66]:
flight.isnull().sum()

FL_DATE                0
OP_CARRIER             0
OP_CARRIER_FL_NUM      0
ORIGIN                 0
DEST                   0
CRS_DEP_TIME           0
DEP_TIME               0
is_delay               0
TAXI_OUT               0
WHEELS_OFF             0
WHEELS_ON              0
TAXI_IN                0
CRS_ARR_TIME           0
ARR_TIME               0
ARR_DELAY              0
CANCELLED              0
CANCELLATION_CODE      0
DIVERTED               0
CRS_ELAPSED_TIME       0
ACTUAL_ELAPSED_TIME    0
AIR_TIME               0
DISTANCE               0
CARRIER_DELAY          0
WEATHER_DELAY          0
NAS_DELAY              0
SECURITY_DELAY         0
LATE_AIRCRAFT_DELAY    0
dtype: int64

In [69]:
flight.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,is_delay,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY


# visualization

In [67]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.countplot(x=flight['DEP_DELAY'])

ValueError: ignored

In [42]:
flight.groupby('DEP_DELAY').size()/len(flight)

is_delay
0.0    0.855648
1.0    0.144352
dtype: float64

In [45]:
#membuat plot dari data kita dapat menggunakan fungsi plot dari library matplotlib
day = flight['FL_DATE'].values
delay  = flight['DEP_DELAY'].values
 
 
plt.figure(figsize=(15,5))
plt.plot(day, delay)
plt.title('Temperature average',
          fontsize=20);

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f09f2cd2dd0> (for post_execute):


OverflowError: ignored

OverflowError: ignored

<Figure size 1080x360 with 1 Axes>

In [19]:
#merubah data kita menjadi format yang dapat diterima oleh model. Fungsi di bawah menerima sebuah series/atribut kita yang telah di konversi menjadi tipe numpy, lalu mengembalikan label dan atribut dari dataset dalam bentuk batch.

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

In [20]:
# arsitektur model gunakan 2 buah layer LSTM. Ketika menggunakan 2 buah layer LSTM, perhatikan bahwa layer pertama harus memiliki parameter return_sequences yang bernilai True.
train_set = windowed_dataset(temp, window_size=60, batch_size=100, shuffle_buffer=1000)
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
])

In [ ]:
model.summary()

In [22]:
optimizer = tf.keras.optimizers.SGD(lr=1.0000e-04, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set,epochs=100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
15/15 [==============================] - 6s 104ms/step - loss: 25.6075 - mae: 26.1075
Epoch 2/100
15/15 [==============================] - 2s 99ms/step - loss: 25.4903 - mae: 25.9903
Epoch 3/100
15/15 [==============================] - 2s 98ms/step - loss: 25.4018 - mae: 25.9018
Epoch 4/100
15/15 [==============================] - 2s 100ms/step - loss: 25.3546 - mae: 25.8546
Epoch 5/100
15/15 [==============================] - 2s 103ms/step - loss: 25.3184 - mae: 25.8184
Epoch 6/100
15/15 [==============================] - 2s 100ms/step - loss: 25.2846 - mae: 25.7846
Epoch 7/100
15/15 [==============================] - 2s 104ms/step - loss: 25.2500 - mae: 25.7500
Epoch 8/100
15/15 [==============================] - 2s 100ms/step - loss: 25.2114 - mae: 25.7114
Epoch 9/100
15/15 [==============================] - 2s 102ms/step - loss: 25.1647 - mae: 25.6647
Epoch 10/100
15/15 [==============================] - 2s 99ms/step - loss: 25.1120 - mae: 25.6120
Epoch 11/100
15/15 [==